In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Data paths
train_dir = (
    "D:\\21881A12D1\\Coding Raja\\Task2\\fruits-360_dataset\\fruits-360\\Training"
)
test_dir = "D:\\21881A12D1\\Coding Raja\\Task2\\fruits-360_dataset\\fruits-360\\Test"

# Data preprocessing
img_size = (100, 100)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

# Load pre-trained model and fine-tune
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(100, 100, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Unfreeze some layers of the base model and fine-tune
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5), loss="categorical_crossentropy", metrics=["accuracy"]
)
history_fine = model.fit(
    train_generator, epochs=10, validation_data=validation_generator
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

# Confusion matrix and classification report
y_pred = np.argmax(model.predict(test_generator), axis=1)
y_true = test_generator.classes

print(
    classification_report(
        y_true, y_pred, target_names=test_generator.class_indices.keys()
    )
)
conf_matrix = confusion_matrix(y_true, y_pred)


# Visualization
def plot_confusion_matrix(cm, classes):
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2
    for i, j in np.ndindex(cm.shape):
        plt.text(
            j,
            i,
            format(cm[i, j], "d"),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")


plt.figure(figsize=(12, 8))
plot_confusion_matrix(conf_matrix, classes=test_generator.class_indices.keys())
plt.show()